In [449]:
import mysql.connector
import numpy as np
import json
import math
from tqdm import tqdm
cnx = mysql.connector.connect(user='ght',
                              host='roctbb.net',
                              database='ghtorrent')
cursor = cnx.cursor()

In [460]:
HH_skill = ['ABAP', 'ASP', 'Assembly', 'C#', 'C++', 'CMake', 'CSS', 'Clojure', 'CoffeeScript', 'ColdFusion', 'Delphi', 'FreeMarker', 'GLSL', 'Go', 'Groovy', 'HTML', 'Handlebars', 'Java', 'JavaScript', 'Kotlin', 'Lua', 'Nginx', 'Objective-C', 'PHP', 'Perl', 'PowerShell', 'Puppet', 'Python', 'Ruby', 'SQL', 'Smarty', 'Swift', 'TypeScript', 'Visual Basic', 'XML', 'XSLT']

In [463]:
import json
import codecs

def sv(obj,p,encoding="utf8"):
    with codecs.open(p,"a",encoding=encoding) as f:
        json.dump(obj,f,indent=1,ensure_ascii=0)

In [456]:
def get_user_experience(user_id, language):
    # пробежаться по проектам юзера на этом языке, посмотреть на все коммиты, +
    # выбрать самый первый(N) и самый последний(K) день коммита на этом языке,(учитывай год!!!)+
    # посчитать количество дней(P), когда чувак коммитил+
    # если P/(K - N) >= 1/2Pi, то записываем log(K - N), в противном случае записываем log(P * 2Pi)+
    cursor.execute("select * from projects where owner_id='%s' and language = '%s';" %(user_id,language))
    userLang = cursor.fetchall()
    userLang = sorted([i[6] for i in userLang]) 
    K_N = (userLang[-1].day+userLang[-1].month*30+userLang[-1].year*365) - (userLang[0].day+userLang[0].month*30+userLang[0].year*365) + 1
    man = [(q.day,q.year) for q in userLang]
    commitDays = len(list(set(map(tuple, man))))
    
    if commitDays/float(K_N) >= 1/2*math.pi:
            return round(math.log(K_N),4)
    return round(math.log(commitDays*2*math.pi),4)

In [461]:
main_data = {}
unique_skills = np.array([])
for user in tqdm(range(1, 10)):
    
    cursor.execute("SELECT * FROM projects where owner_id='%s';" %user)
    user_projects = cursor.fetchall()
    
    if(len(user_projects)==0 or len(user_projects[0])<=5):
        continue
    ####[0] - {language: user experience in language}
    ####[1] - count user followers
    ####[2] - count user wathers
    ####[3] - github link from 
    main_data[user] = [{}, 0, 0, ''] # вот здесь
    #++++++
    # Антон, пожалуйста, не забудь добавить в main_data дополнитульный параметр
    # с ссылкой на настоящий профиль гитхаб юзера
    #
#     print(user_projects,'\n\n\n')   
    user_skills = np.unique([i[5] for i in user_projects if i[5] != None])

    user_link = "https://github.com/" + user_projects[1][1].split('/')[4]
    main_data[user][3] = user_link 
    
    for skill in user_skills:
        if skill in HH_skill:
            main_data[user][0][skill] = get_user_experience(user, skill)
    ### what we heed from this shit 
    ###
    cursor.execute("SELECT user_id FROM followers where user_id='%s';" %user)
    main_data[user][1] = len(cursor.fetchall())
#     print(main_data[user][1])
    cursor.execute("SELECT user_id FROM watchers where user_id='%s';" %user)
    main_data[user][2] = len(cursor.fetchall())
    
    
    unique_skills = np.unique(np.append(unique_skills, user_skills))

100%|██████████| 9/9 [00:10<00:00,  1.21s/it]


In [130]:
for user in columns.items():
    print(user[0],'\n',user[1],'\n')

projects 
 ['id', 'url', 'owner_id', 'name', 'description', 'language', 'created_at', 'forked_from', 'deleted', 'updated_at'] 

watchers 
 ['repo_id', 'user_id', 'created_at'] 

followers 
 ['follower_id', 'user_id', 'created_at'] 

project_members 
 ['repo_id', 'user_id', 'created_at', 'ext_ref_id'] 

issue_comments 
 ['issue_id', 'user_id', 'comment_id', 'created_at'] 

project_commits 
 ['project_id', 'commit_id'] 

repo_labels 
 ['id', 'repo_id', 'name'] 

repo_milestones 
 ['id', 'repo_id', 'name'] 

commit_comments 
 ['id', 'commit_id', 'user_id', 'body', 'line', 'position', 'comment_id', 'created_at'] 

pull_requests 
 ['id', 'head_repo_id', 'base_repo_id', 'head_commit_id', 'base_commit_id', 'pullreq_id', 'intra_branch'] 

issue_events 
 ['event_id', 'issue_id', 'actor_id', 'action', 'action_specific', 'created_at'] 

pull_request_commits 
 ['pull_request_id', 'commit_id'] 

organization_members 
 ['org_id', 'user_id', 'created_at'] 

pull_request_history 
 ['id', 'pull_request